# 4. 단어 추출 word extraction

## 4-a. 단어 추출을 하는 이유
- 커뮤니티 text는 신조어, 은어가 많다.
- 토큰화를 하기 위해선, 신조어와 은어를 추출해야 한다.

## 4-b. 단어 추출 계획
1. text로만 이루어진 txt 파일 만들기
2. soynlp 의 WordExtractor 를 이용하여 단어 추출
3. 단어와 수치 데이터를 csv 파일로 저장

## 4-c. 단어 추출 진행

### 4-c-1. 파일 불러오기

In [1]:
import pandas as pd
df = pd.read_csv("3_spaced_ecopro_text.csv", encoding="utf-8")
df.head(3)

,community,gall_id,search_keyword,number,date_created,time_created,author,is_reply,text_length,spaced_text
0,dcinside,snp500,에코,832014,2023-10-27,15:50:34,ㅇㅇ(58.78),0,56,국연 에코프로 퍼이상 투자해야한다 전국민 자산증발에 앞서야 추후 글로벌 시장에 조선...
1,dcinside,snp500,에코,831598,2023-10-26,21:13:32,슨붕이(106.101),0,11,에코푸로 손절 ㄷㄷㄷ
2,dcinside,snp500,에코,831598,2023-10-26,21:14:04,모든주식을소유하라,1,9,알빠노? 캬ㅋㅋㅋ


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108945 entries, 0 to 108944
Data columns (total 10 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   community       108945 non-null  object
 1   gall_id         108945 non-null  object
 2   search_keyword  108945 non-null  object
 3   number          108945 non-null  int64 
 4   date_created    108945 non-null  object
 5   time_created    108945 non-null  object
 6   author          108945 non-null  object
 7   is_reply        108945 non-null  int64 
 8   text_length     108945 non-null  int64 
 9   spaced_text     108945 non-null  object
dtypes: int64(3), object(7)
memory usage: 8.3+ MB


### 4-c-2. corpus를 list 형태로 만든다

In [3]:
corpus = df['spaced_text'].tolist()

### 4-c-3. word_extractor 학습시키기

In [4]:
from soynlp.word import WordExtractor

word_extractor = WordExtractor(min_frequency=100,
    min_cohesion_forward=0.05, 
    min_right_branching_entropy=0.0
)
word_extractor.train(corpus) # list of str or like
words = word_extractor.extract()

training was done. used memory 0.471 Gbse memory 0.536 Gb
all cohesion probabilities was computed. # words = 3188
all branching entropies was computed # words = 86045
all accessor variety was computed # words = 86045


In [5]:
words['주식']

Scores(cohesion_forward=0.42504616417659896, cohesion_backward=0.14313880126182965, left_branching_entropy=4.545067605105425, right_branching_entropy=4.081748787538966, left_accessor_variety=256, right_accessor_variety=216, leftside_frequency=5064, rightside_frequency=363)

In [6]:
len(words)

2436

### 4-c-4. word extraction 결과 출력

In [7]:
# 결과 출력

import math

def word_score(score):
    return (score.cohesion_forward * math.exp(score.right_branching_entropy))

print('단어   (빈도수, cohesion, branching entropy, word_score)\n')
for word, score in sorted(words.items(), key=lambda x:word_score(x[1]), reverse=True):
    print('%s     (%d, %.3f, %.3f, %.3f)' % (
            word, 
            score.leftside_frequency, 
            score.cohesion_forward,
            score.right_branching_entropy,
            word_score(score)
            )
         )

단어   (빈도수, cohesion, branching entropy, word_score)

프로     (1380, 0.838, 5.206, 152.870)
에코프로     (38919, 0.847, 4.856, 108.831)
진짜     (3871, 0.749, 4.967, 107.619)
으로     (118, 0.492, 5.305, 99.035)
존나     (2792, 0.817, 4.700, 89.799)
차전지     (3278, 0.770, 4.611, 77.435)
에코호로     (9496, 0.529, 4.899, 71.006)
에코프로비엠     (3993, 0.574, 4.785, 68.729)
까지     (612, 0.570, 4.787, 68.399)
공매도     (2454, 0.647, 4.595, 64.064)
ㅋㅋㅋ     (17673, 0.840, 4.299, 61.864)
ㄹㅇ     (2713, 0.988, 4.035, 55.845)
테슬라     (1562, 0.777, 4.234, 53.586)
근데     (2183, 0.768, 4.238, 53.144)
호로     (539, 0.321, 5.098, 52.527)
닥버스     (489, 0.864, 4.079, 51.062)
때문에     (441, 0.562, 4.486, 49.866)
에코프로가     (1547, 0.394, 4.799, 47.852)
에코프로는     (2075, 0.424, 4.711, 47.178)
갑자기     (296, 0.806, 4.036, 45.580)
초전도체     (1032, 0.758, 4.040, 43.060)
했는데     (394, 0.502, 4.415, 41.473)
너무     (1422, 0.619, 4.197, 41.108)
결국     (691, 0.539, 4.302, 39.822)
ㅠㅠ     (1080, 0.930, 3.751, 39.594)
간다     (1328, 0.761, 3.936

### 4-c-5. threshold 판단
- 눈으로 직접 살펴보며, 무의미한 결과를 거를 word_score의 threshold를 정한다
- 살펴본 결과. word_score >= 0.440 인 값만 유의미한 데이터로 간주한다
- 결과 dict : scored_words

In [8]:
threshold = 0.440
# threshold 를 기준으로 dictionary 생성
scored_words = {word: word_score(score) for word, score in words.items() if word_score(score) >= threshold}

print(scored_words)

{'현재': 9.082032482041006, '없어': 0.6519314429717985, '예정': 2.4887060550335596, '결정': 0.5909473600015959, '바로': 19.096550234938423, '않음': 2.3427231173959893, '이거': 6.363565882233241, '입갤': 0.7531208314464387, '잔고': 10.064942302244136, '버핏': 0.5246407845865341, '중에': 7.195212771902769, 'ㅅㄱ': 0.5000813757794467, '반등': 7.910333516698855, '지수': 3.693186279123252, '하면': 7.643909292032921, '모든': 2.6095537648799505, '수익': 5.120285962974181, '서울': 1.275661286342116, '살까': 0.5654709806945298, '차전': 0.6081498315253084, '당장': 4.927767585253922, '해도': 9.459921613187324, '운지': 8.735395013733898, '형님': 8.625873021419231, '조정': 4.926567510929233, '할거': 0.5252348928769695, '흐름': 2.8801263096422467, '진입': 1.4398251439556116, '놈이': 6.050663344552214, '층에': 4.2982725978777925, '흑우': 1.5887208202654355, '경쟁': 0.6261261261261263, '제가': 2.0057161601393365, '증가': 0.887844733699008, '갈때': 1.7227236938066837, '제일': 7.320525255741998, '졸업': 2.328358208955224, '인데': 9.62023718559272, '높음': 1.3982419879883012, '맨날'

In [9]:
len(scored_words)

1277

## 4-d. 단어 정제
1. 영어 포함된 단어 정제
  - 특수한 경우이므로, 프로세스를 나누어 진행한다
2. 조사 포함된 단어 제거
  - "sk텔레콤에" 처럼 조사가 붙은 단어들을 제거한다.
  - 단어 끝에 붙은 조사 처리 : "은", "는", "으로", "이다", "합니다", "였다", "에", "와", "과"
  - list를 뽑아보고, 예외로 처리할 게 있는지 판단한다.
  - "이/가"는 "시가" 같은 멀쩡한 단어를 쪼갤 수 있으니, 진행하지 않거나 주의한다.
    

### 4-d-1. 단어 정제 : 영어가 포함된 단어
- 결과 : scored_words_2

In [23]:
import re

def contains_english(key):
    # 키 값에 영어 알파벳이 하나라도 포함되어 있는지 확인
    return bool(re.search(r'[a-zA-Z]', key))

# 영어 알파벳이 포함된 키들만 추출
english_included_keys = [key for key in scored_words.keys() if contains_english(key)]

print(english_included_keys)
# for key, value in scored_words.items():
#     if contains_english(key):
#         print(f"{key}: {value}")

['vs', 'jp', 'me', 'lg', 'co', 'pe', 'no', 'ht', 'dc', 'ai', 'st', 'ms', 'et', 'sk', 'gm', 'vi', 'msc', 'cfd', 'per', 'txt', 'htt', 'jpg', 'sdi', 'nav', 'dci', 'com', 'etf', 'boa', 'lg화', 'msci', 'dcin', 'boar', 'sk이노', 'http', 'nave', 'lg화학', '삼성sdi', 'https', 'naver', 'dcins', 'board', 'dcinsi', 'dcinsid', 'dcinside']


- sk이노, lg화학, 삼성sdi, etf 만 남긴다

In [14]:
eng_filter = ['vs', 'jp', 'me', 'lg', 'co', 'pe', 'no', 'ht', 'dc', 'ai', 'st', 'ms', 'et', 'sk', 'gm', 'vi', 'msc', 
              'cfd', 'per', 'txt', 'htt', 'jpg', 'sdi', 'nav', 'dci', 'com', 'boa', 'lg화', 'msci', 'dcin', 
              'boar', 'http', 'nave', 'https', 'naver', 'dcins', 'board', 'dcinsi', 'dcinsid', 'dcinside']

In [15]:
# scored_words 에 적용
def remove_matching_keys(dictionary, key_list):
    return {k: v for k, v in dictionary.items() if k not in key_list}

# 함수 적용
scored_words_2 = remove_matching_keys(scored_words, eng_filter)
len(scored_words_2)

1237

### 4-d-2. 단어 정제 : 조사가 포함된 단어

In [17]:
scored_words_2

{'현재': 9.082032482041006,
 '없어': 0.6519314429717985,
 '예정': 2.4887060550335596,
 '결정': 0.5909473600015959,
 '바로': 19.096550234938423,
 '않음': 2.3427231173959893,
 '이거': 6.363565882233241,
 '입갤': 0.7531208314464387,
 '잔고': 10.064942302244136,
 '버핏': 0.5246407845865341,
 '중에': 7.195212771902769,
 'ㅅㄱ': 0.5000813757794467,
 '반등': 7.910333516698855,
 '지수': 3.693186279123252,
 '하면': 7.643909292032921,
 '모든': 2.6095537648799505,
 '수익': 5.120285962974181,
 '서울': 1.275661286342116,
 '살까': 0.5654709806945298,
 '차전': 0.6081498315253084,
 '당장': 4.927767585253922,
 '해도': 9.459921613187324,
 '운지': 8.735395013733898,
 '형님': 8.625873021419231,
 '조정': 4.926567510929233,
 '할거': 0.5252348928769695,
 '흐름': 2.8801263096422467,
 '진입': 1.4398251439556116,
 '놈이': 6.050663344552214,
 '층에': 4.2982725978777925,
 '흑우': 1.5887208202654355,
 '경쟁': 0.6261261261261263,
 '제가': 2.0057161601393365,
 '증가': 0.887844733699008,
 '갈때': 1.7227236938066837,
 '제일': 7.320525255741998,
 '졸업': 2.328358208955224,
 '인데': 9.620237185

In [18]:
key_list = list(scored_words_2.keys())
len(key_list)

1237

### 4-d-3. 조사 필터에 추가할 단어 고르기
- 조사 하나하나 테스트해본다
- "은/는", "로", "다", "에", "와/과", "이/가", "까지", "부터", "처럼", "도", "을/를", "하", "들"
- 결과 : scored_words_3

In [119]:
# 특정 문자열로 끝나는 단어 리턴
def find_endswith(input_list, endswith):
    result = [item for item in input_list if item.endswith(endswith)]
    return result
find_endswith(key_list, "들")

['형들',
 '남들',
 '다들',
 '흔들',
 '니들',
 '놈들',
 '애들',
 '세력들',
 '새끼들',
 '병신들',
 '틀딱들',
 '기관들',
 '사람들',
 '형제들',
 '주주들',
 '주식들',
 '개미들',
 '종목들',
 '아줌마들',
 '이새끼들']

In [120]:
filter_은는 = ['내일은', '지금은', '오늘은', '애들은', '금양은', '주식은', '사람은', '포홀은', '비엠은', '종목은', '신성은', 
               '만원은', '국장은', '새끼들은', '코스닥은', '사람들은', '에코프로같은', '에코프로비엠은', 
               '에코는', '주가는', '올리는', '생각하는', '차전지는', '들어가는', '테슬라는', '에코호로는', '에코프로는']
filter_로다에 = ['프로', '호로', '으로', '앞으로', '주식으로', '생각보다', '모르겠다', '떨어진다', '감사합니다', '에코프로보다',
                 '코갤에', '고점에', '만원에', '에코에', '시장에', '때문에', '주식에', '국장에', '이번에', '하루에',  
                 '최근에', '엘앤에', '작년에', '아침에', '나중에', '만원대에', '에코프로에', '에코호로에']
filter_이 = ['놈이', '것이', '네이', '억이',  '중이', '말이', '신이',  '답이', '장이', '돈이', '시총이', '수익이', 
               '세력이', '실적이', '포홀이', '애들이', '사람이', '삼전이', '신성이', '비엠이', '조정이', '국장이', 
               '병신이', '만원이', '가격이', '시장이', '개인이', '고점이', '지금이', '매출이', '거품이', '생각이',
               '기업이', '놈들이', '기관이', '종목이', '니들이', '오늘이', '영업이', '수급이', '주식이', '금양이',
               '상승이', '느낌이', '가능성이', '개미들이', '미친듯이', '코스닥이', '사람들이', '새끼들이', '에코프로비엠이']
filter_가까지부터처럼을를 = ['드가', '미래가', '새끼가', '자체가', '회사가', '에코가', '개미가', '주가가', '모두가', '이유가', '테슬라가',
                    '차전지가', '초전도체가', '에코프로가', '에코호로가', '까지', '만까지', '년까지', 
                    '부터', '월부터', '처럼', '에코프로처럼', '돈을', '주식을', '에코프로를']
filter_etc = ['에코프', '에코프로로', 'ㅋㅋㅋ코프로', '엔비', '에코프로비', '엔비디', '테슬', '초전', '차전', '갑자',
              '에코프로도', '에코프로의', '에코프로만', '에코프로랑', '에코프로나', '에코도', 'ㅋㅋㅋ캬ㅋㅋㅋ', '공매', '에코랑',
              '병신들', '병신아', '병신들아', '병신새끼', '개병신', '이새끼들', '저새끼', '이새끼', '셀트리', '곱버', 
              '익절하', '궁금하', '감사합니', '레인보', '생각하', '세력들', '새끼들', '병신들', '틀딱들', '기관들', '사람들', 
              '형제들', '주주들', '주식들', '개미들', '종목들', '아줌마들', '이새끼들']

### 4-d-4. 조사 필터 추가 및 적용

In [125]:
particle_filter = []
particle_filter.extend(filter_은는)
particle_filter.extend(filter_로다에)
particle_filter.extend(filter_이)
particle_filter.extend(filter_가까지부터처럼을를)
particle_filter.extend(filter_etc)

# 함수 적용
scored_words_3 = remove_matching_keys(scored_words_2, particle_filter)
len(scored_words_3)

1061

### 4-d-5. word_list 출력

In [124]:
def print_sorted_dict(my_dict):
    # 사전을 value 기준 내림차순으로 정렬
    sorted_dict = dict(sorted(my_dict.items(), key=lambda item: item[1], reverse=True))
    
    # 정렬된 사전의 각 원소를 한 줄에 하나씩 출력
    for key, value in sorted_dict.items():
        print(f"{key}: {value}")

# 함수 호출
print_sorted_dict(scored_words_3)

에코프로: 108.83068376262308
진짜: 107.61890055798742
존나: 89.79863952326305
차전지: 77.43544798577665
에코호로: 71.0060236755185
에코프로비엠: 68.72864238033726
공매도: 64.0641958398311
ㅋㅋㅋ: 61.8643222997944
ㄹㅇ: 55.84515868430428
테슬라: 53.58581074607017
근데: 53.14390891855639
닥버스: 51.0616409785589
갑자기: 45.580075203268535
초전도체: 43.0597013443739
했는데: 41.472696757728656
너무: 41.108339894234774
결국: 39.821672050442054
ㅠㅠ: 39.59444310089299
간다: 39.00017787738813
지금: 38.307994796383966
씨발: 37.36668987474884
코스닥: 37.124352859996556
비엠: 36.166137441833996
계속: 35.10414813365147
라고: 33.814716298472156
만원: 33.42033133828582
솔직히: 33.317103891561366
슬슬: 33.19213477947861
혼자: 33.07597855382817
같은데: 32.84703517003681
된다: 30.915233387167497
회사: 30.08423853532345
셀트리온: 29.81297820222059
ㅋㅋ: 29.625122004560797
카카오: 29.455271564787758
배터리: 29.233829054465886
ㄷㄷ: 29.175889111160455
오늘: 29.11223480661084
바이오: 28.444688682437626
반도체: 28.269423119105994
ㅡㅡ: 28.1917439908377
ㅇㅇ: 28.053948682690883
누가: 28.013079870039636
항상: 27.9698161

#### 결과 확인 Tool

In [123]:
# 특정 문자열로 끝나는 단어 리턴
def find_endswith(input_list, endswith):
    result = [item for item in input_list if item.endswith(endswith)]
    return result
find_endswith(list(scored_words_3.keys()), "들")

['형들', '남들', '다들', '흔들', '니들', '놈들', '애들']

### 4-d-6. 향후 nomalization을 위한 저육화 리스트 만들어두기
- 에코프로 : 에코호로, 에코프로비엠, 에코호로비엠, 에코비엠, 에코후로, 에코형제
- 시발 : ㅅㅂ, 씨발, 씨발년, 씨발련, 시벌, 씨벌, 시불, 씨불
- 병신 : ㅂㅅ, ㅄ, 븅신
- 새끼 : 새끼들, 새기, 세끼

## 4-e. 단어 리스트를 파일로 저장

### 4-e-1. 파일로 저장하기

In [127]:
# 파일로 저장
filename = "4_ecopro_word_list.txt"
words = scored_words_3
with open(filename, "w", encoding="utf-8") as file:
    for word in words:
        file.write(word + "\n")

print("파일 저장 완료.")

파일 저장 완료.


### 4-e-2. 불러와서 확인하기

In [128]:
# 파일로부터 문자열 리스트 불러오기
with open(filename, "r", encoding="utf-8") as file:
    word_list = [line.rstrip('\n') for line in file]

print("불러온 문자열 리스트:")
print(word_list)

불러온 문자열 리스트:
['현재', '없어', '예정', '결정', '바로', '않음', '이거', '입갤', '잔고', '버핏', '중에', 'ㅅㄱ', '반등', '지수', '하면', '모든', '수익', '서울', '살까', '당장', '해도', '운지', '형님', '조정', '할거', '흐름', '진입', '층에', '흑우', '경쟁', '제가', '증가', '갈때', '제일', '졸업', '인데', '높음', '맨날', '매수', '장기', '털고', '잡고', '거의', '병신', '좋아', '구조', '기관', '길게', '피자', '성장', '결국', '박살', '뉴스', '별로', '샀어', '특징', '대한', '않나', '맞냐', '소재', '난리', '찍고', '본다', '몇개', '랠리', '양극', '믿는', '심리', '갔다', '똑같', '방어', '회복', '규모', '밸류', '빠진', '맞음', '매매', '바닥', '되냐', '수준', '올해', '이런', '방법', '너네', '축하', '누구', '무슨', '관심', '해서', '벌써', '섹터', '엘앤', '큐ㅠ', '급락', '언제', '했다', '에코', '테마', '추매', '정리', '억원', '부자', '쉽게', '그냥', '달러', '방금', '없다', '하락', '봐야', '승리', '되나', '맞다', 'ㅆㅂ', '너무', '확률', '직원', '소리', '싸게', '벌고', '연상', '많음', '재미', '허허', '물린', '분명', '은행', 'ㅅㅂ', '머리', '캬ㅋ', '되지', '슈카', '다음', '걱정', '적정', '형들', '앞으', '구간', '살껄', '상승', '넘게', '삼성', '포홀', '국내', '영익', '그런', '산업', '보고', '미래', '남들', '굳이', '오를', '분기', '크게', 'ㅎㅎ', '팔면', '진짜', '확정', '두산', '선동', '유증', '전부', '백만', '버스', '저점', '호